In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from tqdm import tqdm
from array import array
from utility import *

# Adaptive-FAM
Encode large amount multi-resolution micro-blocks into micro-models using function approximation. Using Flame dataset as a input, same process applies to other datasets. Multi-resolution LOD is set as 4. Function approximation degree is set 2.

## 1. Create Micro-blocks
Partition input dataset into multi-resolution micro-blocks with 4 levels of detail (LOD)

### 1.1 Fetch the large-scale dataset
Flame dataset (1201x1201x1201 resolution in float32 numpy array format, flame_1201x1201x1201.npy), from https://drive.google.com/file/d/1fFB5LpRkiVJIZ47Ckc0VCGYM_M4EU3ZX/view?usp=sharing

In [ ]:
a_load = np.load('flame_1201x1201x1201.npy')

### 1.2 Partition the input dataset and downsample into micro-blocks for each LOD

### Level 1: block size: 76x76x76; Number: 16x16x16

In [ ]:
edge_number = 16
edge_size = 76
for x in tqdm (range(edge_number), desc="Saving partitioned blocks..."):
    for y in range(edge_number):
        for z in range(edge_number):
            index = x*edge_number*edge_number + y*edge_number + z
            x_start = x*75
            x_end = x_start + 75
            y_start = y*75
            y_end = y_start + 75
            z_start = z*75
            z_end = z_start + 75
            data = a_load[x_start:x_end + 1, y_start:y_end + 1, z_start:z_end + 1]
            data = data.reshape(76*76*76,)
            saveToMfaXyz(data, "level_1_flame_downsampled/" + str(index + 8 + 64 + 512) + ".xyz")

### Level 2: block size: 151x151x151; Number: 8x8x8

In [ ]:
edge_number = 8
edge_size = 151
for x in tqdm (range(edge_number), desc="Saving partitioned blocks..."):
    for y in range(edge_number):
        for z in range(edge_number):
            index = x*edge_number*edge_number + y*edge_number + z
            x_start = x*150
            x_end = x_start + 150
            y_start = y*150
            y_end = y_start + 150
            z_start = z*150
            z_end = z_start + 150
            data = a_load[x_start:x_end + 1, y_start:y_end + 1, z_start:z_end + 1][0:151:2, 0:151:2, 0:151:2]
            data = data.reshape(76*76*76,)
            saveToMfaXyz(data, "level_2_flame_downsampled/" + str(index + 8 + 64) + ".xyz")

### Level 3: block size: 301x301x301; Number: 4x4x4

In [ ]:
edge_number = 4
edge_size = 301
for x in tqdm (range(edge_number), desc="Saving partitioned blocks..."):
    for y in range(edge_number):
        for z in range(edge_number):
            index = x*edge_number*edge_number + y*edge_number + z
            x_start = x*300
            x_end = x_start + 300
            y_start = y*300
            y_end = y_start + 300
            z_start = z*300
            z_end = z_start + 300
            data = a_load[x_start:x_end + 1, y_start:y_end + 1, z_start:z_end + 1][0:301:4, 0:301:4, 0:301:4]
            data = data.reshape(76*76*76,)
            saveToMfaXyz(data, "level_3_flame_downsampled/" + str(index + 8) + ".xyz")

### Level 4: block size: 601x601x601; Number: 2x2x2

In [ ]:
edge_number = 2
edge_size = 601
for x in tqdm (range(edge_number), desc="Saving partitioned blocks..."):
    for y in range(edge_number):
        for z in range(edge_number):
            index = x*edge_number*edge_number + y*edge_number + z
            x_start = x*600
            x_end = x_start + 600
            y_start = y*600
            y_end = y_start + 600
            z_start = z*600
            z_end = z_start + 600
            data = a_load[x_start:x_end + 1, y_start:y_end + 1, z_start:z_end + 1][0:601:8, 0:601:8, 0:601:8] # 76x76x76
            data = data.reshape(76*76*76,)
            saveToMfaXyz(data, "level_4_flame_downsampled/" + str(index) + ".xyz")

## 2. Apply Adaptive-FAM

Encode each micro-block into micro-model with suitable number of control points (NCP) for given error bound which is set as 0.00001 of root mean square error (RMS). The functional approximation degree is set to 2.

In [ ]:
encoder = '../../mfa_utility/build/src/fixed/fixed'

In [ ]:
all_suitable_num_ctrlpts_lists = []

### 2.1 Do in-level encoding for the lowest LOD (Level 4). Only needs to handle 8 micro-blocks

In [ ]:
# Do in-level encoding for all micro-blocks
for m in tqdm (list(range(0,8,1))):
    os.system('mkdir level_4/block_' + str(m))
    for n in list(range(3,77,1)):
        os.system(encoder + ' -m 3 -d 4 -f level_4_flame_downsampled/' + str(m) + 
                  '.xyz -i general -n 76 -v ' + str(n) + ' -q 2 >/dev/null 2>&1')
        os.system('mv test.mfab level_4/block_' + str(m) + '/' + str(n) + '.mfab')
        os.system('rm approx.out')
    os.system('mv error.txt level_4/block_' + str(m))

In [ ]:
# Collect errors resulting from in-level encoding for each micro-block
block_idx = list(range(0,8,1))
error_list = []
for i in block_idx:
    file1 = open('level_4/block_' + str(i) + '/error.txt', 'r')
    Lines = file1.readlines()
    count = 0
    error = []
    num_ctrlpts = []
    for line in Lines:
        num_ctrlpts.append(count + 3)
        count += 1
        line = line[:-1]
        error.append(float(line))
    error_list.append(error)
    file1.close()

# Find complex micro-blocks
suitable_num_ctrlpts_list = getSuitableNumOfCtrlpts(error_list, num_ctrlpts, 0.00001)
complex_mb_ctrlpts_list = [] # only list number of control points > 3, 3 is the minimal for degree 2
complex_mb_index_list = []
simple_mb_index_list = []
for i in range(len(suitable_num_ctrlpts_list)):
    if (suitable_num_ctrlpts_list[i] > 3):
        complex_mb_ctrlpts_list.append(suitable_num_ctrlpts_list[i])
        complex_mb_index_list.append(i)
    else:
        simple_mb_index_list.append(i)
    
print("Level 4 complex micro-blocks index: ", complex_mb_index_list)
print("Level 4 simple micro-blocks index: ", simple_mb_index_list)

In [ ]:
all_suitable_num_ctrlpts_lists.append(suitable_num_ctrlpts_list)

### 2.2 Do cross-level encoding for the rest level with higher LOD (Level 3, 2, 1)

### Level 3

In [ ]:
# find indices of all micro-block in level 3 covered by complex blocks in level 4
next_level_complex_mb_index_list = getComplexMicroBlockIndices(complex_mb_index_list, 2, 0, 8)

In [ ]:
# Do in-level encoding only for complex micro-blocks
for m in tqdm (next_level_complex_mb_index_list):
    os.system('mkdir level_3/block_' + str(m))
    for n in list(range(3,77,1)):
        os.system(encoder + ' -m 3 -d 4 -f level_3_flame_downsampled/' + str(m) + 
                  '.xyz -i general -n 76 -v ' + str(n) + ' -q 2 >/dev/null 2>&1')
        os.system('mv test.mfab level_3/block_' + str(m) + '/' + str(n) + '.mfab')
        os.system('rm approx.out')
    os.system('mv error.txt level_3/block_' + str(m))

In [ ]:
# Collect errors resulting from in-level encoding for each complex micro-block
block_idx = next_level_complex_mb_index_list
error_list = []
for i in block_idx:
    file1 = open('level_3/block_' + str(i) + '/error.txt', 'r')
    Lines = file1.readlines()
    count = 0
    error = []
    num_ctrlpts = []
    for line in Lines:
        num_ctrlpts.append(count + 3)
        count += 1
        line = line[:-1]
        error.append(float(line))
    error_list.append(error)
    file1.close()

# Find complex micro-blocks
suitable_num_ctrlpts_list = getSuitableNumOfCtrlpts(error_list, num_ctrlpts, 0.00001)
complex_mb_ctrlpts_list = [] # only list number of control points > 3, 3 is the minimal for degree 2
complex_mb_index_list = []
simple_mb_index_list = []
for i in range(len(suitable_num_ctrlpts_list)):
    if (suitable_num_ctrlpts_list[i] > 3):
        complex_mb_ctrlpts_list.append(suitable_num_ctrlpts_list[i])
        complex_mb_index_list.append(i + 8)
    else:
        simple_mb_index_list.append(i + 8)
    
print("Level 3 complex micro-blocks index: ", complex_mb_index_list)
print("Level 3 simple micro-blocks index: ", simple_mb_index_list)

In [ ]:
all_suitable_num_ctrlpts_lists.append(suitable_num_ctrlpts_list)

### Level 2

In [ ]:
# find indices of all micro-block in level 2 covered by complex blocks in level 3
next_level_complex_mb_index_list = getComplexMicroBlockIndices(complex_mb_index_list, 4, 8, 8 + 64)

In [ ]:
# find indices of all micro-block in level 2 not covered by complex blocks in level 3
total_blocks = list(range(72,584,1))
next_level_simple_mb_index_list = []
for i in range(len(total_blocks)):
    if (total_blocks[i] not in next_level_complex_mb_index_list):
        next_level_simple_mb_index_list.append(total_blocks[i])

In [ ]:
# Do in-level encoding only for complex micro-blocks
for m in tqdm (next_level_complex_mb_index_list):
    os.system('mkdir level_2/block_' + str(m))
    for n in list(range(3,77,1)):
        os.system(encoder + ' -m 3 -d 4 -f level_2_flame_downsampled/' + str(m) + 
                  '.xyz -i general -n 76 -v ' + str(n) + ' -q 2 >/dev/null 2>&1')
        os.system('mv test.mfab level_2/block_' + str(m) + '/' + str(n) + '.mfab')
        os.system('rm approx.out')
    os.system('mv error.txt level_2/block_' + str(m))

In [ ]:
# Do encoding for all simple micro-blocks
for m in tqdm (next_level_simple_mb_index_list):
    os.system('mkdir level_2/block_' + str(m))
    os.system(encoder + ' -m 3 -d 4 -f level_2_flame_downsampled/' + str(m) + 
              '.xyz -i general -n 76 -v 3 -q 2 >/dev/null 2>&1')
    os.system('mv test.mfab level_2/block_' + str(m) + '/3.mfab')
    os.system('rm approx.out')
os.system('rm error.txt')

In [ ]:
# Collect errors resulting from in-level encoding for each complex micro-block
block_idx = next_level_complex_mb_index_list
error_list = []
for i in block_idx:
    file1 = open('level_2/block_' + str(i) + '/error.txt', 'r')
    Lines = file1.readlines()
    count = 0
    error = []
    num_ctrlpts = []
    for line in Lines:
        num_ctrlpts.append(count + 3)
        count += 1
        line = line[:-1]
        error.append(float(line))
    error_list.append(error)
    file1.close()

# Find complex micro-blocks
suitable_num_ctrlpts_list = getSuitableNumOfCtrlpts(error_list, num_ctrlpts, 0.00001)
complex_mb_ctrlpts_list = [] # only list number of control points > 3, 3 is the minimal for degree 2
complex_mb_index_list = []
for i in range(len(suitable_num_ctrlpts_list)):
    if (suitable_num_ctrlpts_list[i] > 3):
        complex_mb_ctrlpts_list.append(suitable_num_ctrlpts_list[i])
        complex_mb_index_list.append(next_level_complex_mb_index_list[i])

total_blocks = list(range(72,584,1))
simple_mb_index_list = []
for i in range(len(total_blocks)):
    if (total_blocks[i] not in complex_mb_index_list):
        simple_mb_index_list.append(total_blocks[i])
    
print("Level 2 complex micro-blocks index: ", complex_mb_index_list)
print("Level 2 simple micro-blocks index: ", simple_mb_index_list)

In [ ]:
total_blocks = list(range(72,584,1))
all_suitable_num_ctrlpts_list = []
for i in range(len(total_blocks)):
    if (total_blocks[i] in next_level_complex_mb_index_list):
        for j in range(len(next_level_complex_mb_index_list)):
            if (total_blocks[i] == next_level_complex_mb_index_list[j]):
                all_suitable_num_ctrlpts_list.append(suitable_num_ctrlpts_list[j])
                break
    else:
        all_suitable_num_ctrlpts_list.append(3)
        
all_suitable_num_ctrlpts_lists.append(all_suitable_num_ctrlpts_list)

### Level 1

In [ ]:
# find indices of all micro-block in level 1 covered by complex blocks in level 2
next_level_complex_mb_index_list = getComplexMicroBlockIndices(complex_mb_index_list, 8, 8 + 64, 8 + 64 + 512)

In [ ]:
# find indices of all micro-block in level 1 not covered by complex blocks in level 2
total_blocks = list(range(584,4680,1))
next_level_simple_mb_index_list = []
for i in range(len(total_blocks)):
    if (total_blocks[i] not in next_level_complex_mb_index_list):
        next_level_simple_mb_index_list.append(total_blocks[i])

In [ ]:
# Do in-level encoding only for complex micro-blocks
for m in tqdm (next_level_complex_mb_index_list):
    os.system('mkdir level_1/block_' + str(m))
    for n in list(range(3,77,1)):
        os.system(encoder + ' -m 3 -d 4 -f level_1_flame_downsampled/' + str(m) + 
                  '.xyz -i general -n 76 -v ' + str(n) + ' -q 2 >/dev/null 2>&1')
        os.system('mv test.mfab level_1/block_' + str(m) + '/' + str(n) + '.mfab')
        os.system('rm approx.out')
    os.system('mv error.txt level_1/block_' + str(m))

In [ ]:
# Do encoding for all simple micro-blocks
for m in tqdm (next_level_simple_mb_index_list):
    os.system('mkdir level_1/block_' + str(m))
    os.system(encoder + ' -m 3 -d 4 -f level_1_flame_downsampled/' + str(m) + 
              '.xyz -i general -n 76 -v 3 -q 2 >/dev/null 2>&1')
    os.system('mv test.mfab level_1/block_' + str(m) + '/3.mfab')
    os.system('rm approx.out')
os.system('rm error.txt')

In [ ]:
# Collect errors resulting from in-level encoding for each complex micro-block
block_idx = next_level_complex_mb_index_list
error_list = []
for i in block_idx:
    file1 = open('level_1/block_' + str(i) + '/error.txt', 'r')
    Lines = file1.readlines()
    count = 0
    error = []
    num_ctrlpts = []
    for line in Lines:
        num_ctrlpts.append(count + 3)
        count += 1
        line = line[:-1]
        error.append(float(line))
    error_list.append(error)
    file1.close()

# Find complex micro-blocks
suitable_num_ctrlpts_list = getSuitableNumOfCtrlpts(error_list, num_ctrlpts, 0.00001)
complex_mb_ctrlpts_list = [] # only list number of control points > 3, 3 is the minimal for degree 2
complex_mb_index_list = []
for i in range(len(suitable_num_ctrlpts_list)):
    if (suitable_num_ctrlpts_list[i] > 3):
        complex_mb_ctrlpts_list.append(suitable_num_ctrlpts_list[i])
        complex_mb_index_list.append(next_level_complex_mb_index_list[i])

total_blocks = list(range(584,4680,1))
simple_mb_index_list = []
for i in range(len(total_blocks)):
    if (total_blocks[i] not in complex_mb_index_list):
        simple_mb_index_list.append(total_blocks[i])
    
print("Level 1 complex micro-blocks index: ", complex_mb_index_list)
print("Level 1 simple micro-blocks index: ", simple_mb_index_list)

In [ ]:
total_blocks = list(range(584,4680,1))
all_suitable_num_ctrlpts_list = []
for i in range(len(total_blocks)):
    if (total_blocks[i] in next_level_complex_mb_index_list):
        for j in range(len(next_level_complex_mb_index_list)):
            if (total_blocks[i] == next_level_complex_mb_index_list[j]):
                all_suitable_num_ctrlpts_list.append(suitable_num_ctrlpts_list[j])
                break
    else:
        all_suitable_num_ctrlpts_list.append(3)
        
all_suitable_num_ctrlpts_lists.append(all_suitable_num_ctrlpts_list)

### 2.3 Collect all the micro-models of all 4 levels with suitable NCP into one folder as inputs to renderer

In [ ]:
# put correct mfab blocks to the folder "data_flame_adaptive"
# level 4
for i in range(len(all_suitable_num_ctrlpts_lists[0])):
    shutil.copyfile('level_4/block_' + str(i) + '/' + str(all_suitable_num_ctrlpts_lists[0][i]) + '.mfab', \
                    'data_flame_adaptive/' + str(i) + '.mfab')

In [ ]:
# level 3
for i in range(len(all_suitable_num_ctrlpts_lists[1])):
    shutil.copyfile('level_3/block_' + str(i + 8) + '/' + str(all_suitable_num_ctrlpts_lists[1][i]) + '.mfab', \
                    'data_flame_adaptive/' + str(i + 8) + '.mfab')

In [ ]:
# level 2
for i in range(len(all_suitable_num_ctrlpts_lists[2])):
    shutil.copyfile('level_2/block_' + str(i + 8 + 64) + '/' + str(all_suitable_num_ctrlpts_lists[2][i]) + '.mfab', \
                    'data_flame_adaptive/' + str(i + 8 + 64) + '.mfab')

In [ ]:
# level 1
for i in range(len(all_suitable_num_ctrlpts_lists[3])):
    shutil.copyfile('level_1/block_' + str(i + 8 + 64 + 512) + '/' + str(all_suitable_num_ctrlpts_lists[3][i]) + '.mfab', \
                    'data_flame_adaptive/' + str(i + 8 + 64 + 512) + '.mfab')